# **Roberta Base - Deploying model**

## **Load pre-trained model and save model locally**

In [ ]:
from transformers import AutoModelForSequenceClassification, TFAutoModelForSequenceClassification, AutoTokenizer
from scipy.special import softmax
import pandas as pd
import numpy as np

In [ ]:
# download the model
MODEL = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [ ]:
# save the model
save_dir = "./roberta_base_artifacts"
tokenizer.save_pretrained(save_dir)
model.save_pretrained(save_dir)

## **Load the model from local files**

In [ ]:
class Model:
    """A model class to lead the model and tokenizer"""
    
    def __init__(self) -> None:
        pass
  
    def load_model():
        model = AutoModelForSequenceClassification.from_pretrained(save_dir)
        
        return model

    def load_tokenizer():
        tokenizer = AutoTokenizer.from_pretrained(save_dir)
        
        return tokenizer

## **Make prediction from locally loaded model**

In [ ]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


text = "Good night bitch I hate you😊"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')

output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
print(scores)

In [ ]:
#only top score. 
result = {}
labels = ["Negative", "Neutral", "Positive"]
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
result["label"] = str(labels[ranking[0]])
result["score"] = np.round(float(scores[ranking[0]]), 4)
result

## **Local prediction again. Serialized as json input. Needed for when deployed**

In [ ]:
#make post request
data = {"text": "Hi, Thanks, i'm very sad"}

#fetch text from input
text_input = data['text']


# same script again
text = preprocess(text_input)
encoded_input = tokenizer(text, return_tensors='pt')

output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
print(scores)

-----

# **Create model artifacts**

In [2]:
from ads.model.framework.tensorflow_model import TensorFlowModel
from ads.common.model_metadata import UseCaseType
from ads.common.model_artifact import ModelArtifact
from ads.common.model_export_util import prepare_generic_model
import os

In [3]:
#path to artifacts and conda slug
path_to_artifacts = './model_artifacts_roberta_base_sentiment'
conda_env = 'oci://conda_environment_yolov5@frqap2zhtzbe/conda_environments/cpu/fdf_conda/1.0/fdf_conda'   #this refers to the published conda location (bucket name, namespace)

#create default artifacts
artifact = prepare_generic_model(
    path_to_artifacts, 
    fn_artifact_files_included=False, 
    force_overwrite=True, 
    inference_conda_env=conda_env)

loop1:   0%|          | 0/4 [00:00<?, ?it/s]

# **Change the score.py manually**

In [4]:
##########
##########
########## IMPORTANT. AFTER CREATING ARTIFACT FILESC HANGE PYTHON VERSION in the YAML file
##########
##########  ADD "inference_python_version=3.7.0 to the above prepare_generic_model
##########   

In [ ]:
#copy all files in the model artifacts
!cp -a ./roberta_base_artifacts ./model_artifacts_roberta_base_sentiment

In [ ]:
####
###
##  Output prediction is probably not entirely serialized? 


In [5]:
%%writefile "{path_to_artifacts}/score.py"
import os
import ads
from transformers import AutoModelForSequenceClassification, TFAutoModelForSequenceClassification, AutoTokenizer
import tokenizers
import pandas as pd
import numpy as np
import json
import os
from cloudpickle import cloudpickle
from functools import lru_cache
from scipy.special import softmax

#load model and tokenizer
model_artifacts_folder = "./roberta_base_artifacts"  

def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)



def load_model():
    class DummyModel:
        def __init__(self):
            pass
    return DummyModel()

#loading the model before seemed to fail. NOt sure why. Now loading in the predict.

def predict(data, model=load_model()):
    
    model = AutoModelForSequenceClassification.from_pretrained(model_artifacts_folder)
    tokenizer = AutoTokenizer.from_pretrained(model_artifacts_folder)
    
    #fetch text from input
    text_input = data['text']

    # process text
    text = preprocess(text_input)
    encoded_input = tokenizer(text, return_tensors='pt')
    encoded_input
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    
    #convert scores to json
    scores_list = scores.tolist()
    json_str = json.dumps(scores_list)
    
    return {'prediction': json_str}

Overwriting ./model_artifacts_roberta_base_sentiment/score.py


In [ ]:
#smal test on local predict()

data = {"text": "Hi, Thanks, i'm very sad"}

predict(data)

## **check the artifacts**

In [6]:
#all should be passed
artifact.introspect()

['__pycache__', 'test_json_output.json', '.ipynb_checkpoints', 'runtime.yaml', 'roberta_base_artifacts', 'score.py']


,Test key,Test name,Result,Message
0,runtime_env_path,Check that field MODEL_DEPLOYMENT.INFERENCE_ENV_PATH is set,Passed,
1,runtime_env_python,Check that field MODEL_DEPLOYMENT.INFERENCE_PYTHON_VERSION is set to a value of 3.6 or higher,Passed,
2,runtime_path_exist,Check that the file path in MODEL_DEPLOYMENT.INFERENCE_ENV_PATH is correct.,Passed,
3,runtime_version,Check that field MODEL_ARTIFACT_VERSION is set to 3.0,Passed,
4,runtime_yaml,"Check that the file ""runtime.yaml"" exists and is in the top level directory of the artifact directory",Passed,
5,score_load_model,Check that load_model() is defined,Passed,
6,score_predict,Check that predict() is defined,Passed,
7,score_predict_arg,Check that all other arguments in predict() are optional and have default values,Passed,
8,score_predict_data,"Check that the only required argument for predict() is named ""data""",Passed,
9,score_py,"Check that the file ""score.py"" exists and is in the top level directory of the artifact directory",Passed,


## **Test model from artifacts**

In [7]:
import sys
sys.path.insert(0, path_to_artifacts)
 
# importing load_model() and predict() that are defined in score.py
from score import load_model, predict
 
# Loading the model to memory
_ = load_model()

In [8]:
data = {"text": "Hi, Thanks, i'm very sad"}

predictions_test = predict(data, _)
predictions_test

{'prediction': '[0.8859006762504578, 0.09977766126394272, 0.014321762137115002]'}

## **store in catalog**

In [9]:
# Saving the model artifact to the model catalog. 
catalog_entry = artifact.save(display_name='roberta_base_v1', description='roberta_base_v1', timeout=600)
catalog_entry.id

loop1:   0%|          | 0/5 [00:00<?, ?it/s]

artifact:/tmp/saved_model_7e5a1425-b7dd-4b8b-99d3-af4d9444ca2c.zip


'ocid1.datasciencemodel.oc1.eu-frankfurt-1.amaaaaaangencdya6kby5jvilx7lghnpkd72qiwbh3fhoxog6hadphxscbla'

## **Deploy in the UI**

## **Invoke the model**

In [10]:
import requests
import oci
from oci.signer import Signer

In [11]:
#fdf version mf_final
uri = f"https://modeldeployment.eu-frankfurt-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaangencdya25yroloh5zt65kh7aoisqwmk2ulrt3rrgv7mxytovl2a/predict"
print(uri)

https://modeldeployment.eu-frankfurt-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaangencdya25yroloh5zt65kh7aoisqwmk2ulrt3rrgv7mxytovl2a/predict


In [12]:
# Using Resource principal to authenticate against the model endpoint. Set using_rps=False if you are using the config+key flow. 
using_rps = False

if using_rps: # using resource principal:     
    auth = oci.auth.signers.get_resource_principals_signer()
else: # using config + key: 
    config = oci.config.from_file("~/.oci/config") # replace with the location of your oci config file
    auth = Signer(
        tenancy=config['tenancy'],
        user=config['user'],
        fingerprint=config['fingerprint'],
        private_key_file_location=config['key_file'],
        pass_phrase=config['pass_phrase'])

In [22]:
%%time
import json

#input data
data = {"text": "Hi, Thanks, i'm very positive today."}

#POST request to the model
response = requests.post(uri, json=data, auth=auth)
print(response)
print(json.loads(response.content))

#label 1 = Negative
#Label 2 = Medium
#Label 3 = Positive

<Response [200]>
{'prediction': '[0.001066302414983511, 0.017006639391183853, 0.9819270372390747]'}
CPU times: user 18.9 ms, sys: 2.68 ms, total: 21.6 ms
Wall time: 1.09 s
